In [1]:
# import necessary modules
# reading data
import os
import netCDF4 as nc
# operation data
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
#from mpl_toolkits.basemap import Basemap
from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
# visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
from mpl_toolkits.axes_grid1 import make_axes_locatable, inset_locator
from matplotlib.ticker import AutoMinorLocator
# machine learing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KernelDensity
#import hdbscan
import debacl as dcl
# saving as pdf
#from matplotlib.backends.backend_pdf import PdfPages

In [2]:
#get the file path for loading, data file is under the same dir with the notebook
filename="20121015_00_ecmwf_ensemble_forecast.PRESSURE_LEVELS.EUR_LL10.120.pl.nc"
foldername="ECWMF Datasets"
filepath=os.path.join(os.path.dirname(os.getcwd()),foldername,filename)

# read the raw data and extract the needed data
# exrtact the value of Geopotential under the pressure of 500 hPA in the certain
Pressure_Levels_data = nc.Dataset(filepath,"r")
g = 9.80655
# get all the dimension value
nd_1,nd_2,nd_3,nd_4,nd_5 = Pressure_Levels_data.variables['Geopotential_isobaric'][:].shape
# get the necessary raw data
Geopotential_Isobaric_500 = Pressure_Levels_data.variables['Geopotential_isobaric'][0,:,7,:,:]/g
# reshape the dataset into form of (51,41*101)
Geopotential_Isobaric_500_reshaped = np.reshape(Geopotential_Isobaric_500,(nd_2, nd_4 * nd_5))
# prepare the longitude and latitude value for contour
longitude = Pressure_Levels_data['lon'][:]
latitude = Pressure_Levels_data['lat'][:]
(lon, lat) = np.meshgrid(longitude, latitude)

# use PCA to reduce dimensions under the condition of reaching 80% of all the member infomation
exp_var = 0
n_pc = 0
while exp_var < 0.8:
    n_pc = n_pc + 1
    pca = PCA(n_components = n_pc)
    pca.fit(Geopotential_Isobaric_500_reshaped)
    exp_var = sum(pca.explained_variance_ratio_)

# get the transformed raw data in the dimension-reduced space    
pca_transformed_data = pca.transform(Geopotential_Isobaric_500_reshaped)

#get the time variable
times=Pressure_Levels_data.variables["time"]
#get the time number
arrDateEnd=nc.num2date(times[:],units=times.units)
#get the time in date format
dateEndDate = datetime.date(arrDateEnd[0]).strftime("%d %b %Y")
dateEndMin = datetime.date(arrDateEnd[0]).strftime("%H:%M")
dateStart=datetime.date(arrDateEnd[0])-timedelta(hours=120)
dateStartDate=dateStart.strftime("%d %b %Y")
dateStartMin=dateStart.strftime("%H:%M")

In [3]:
def Get_Subgrid(longitude,latitude,bottomLeft,topRight):
    lonStart=(np.abs(longitude - bottomLeft[0])).argmin()
    lonEnd=(np.abs(longitude - topRight[0])).argmin()
    latStart=(np.abs(latitude - bottomLeft[1])).argmin()
    latEnd=(np.abs(latitude - topRight[1])).argmin()
    
    return lonStart,lonEnd,latStart,latEnd

def Get_Meshgrid(lonStart,lonEnd,latStart,latEnd):
    long=longitude[lonStart:lonEnd+1]
    lati=latitude[latEnd:latStart+1]
    (lon,lat)=np.meshgrid(long,lati)
    
    return lon,lat

def Get_Area_Data(data,lonStart,lonEnd,latStart,latEnd):
    areaData=data[:,latEnd:latStart+1,lonStart:lonEnd+1]
    
    return areaData

def Reshape_New_Data(areaData):
    dim1,dim2,dim3=areaData.shape
    reshapedData=np.reshape(areaData,(dim1,dim2*dim3))
    return reshapedData

def PCA_Run(data):
    exp_var = 0
    n_pc = 0
    while exp_var < 0.8:
        n_pc = n_pc + 1
        pca = PCA(n_components = n_pc)
        pca.fit(data)
        exp_var = sum(pca.explained_variance_ratio_)
        
    return pca


In [4]:
#get the file path for loading, data file is under the same dir with the notebook
filename="20121017_12_ecmwf_forecast.PRESSURE_LEVELS.EUR_LL015.036.pl.nc"
foldername="ECWMF Datasets"
filepath=os.path.join(os.path.dirname(os.getcwd()),foldername,filename)
syn = nc.Dataset(filepath,"r")
#syn.variables
longitude1 = syn['lon'][:]
latitude1 = syn['lat'][:]
(lon1, lat1) = np.meshgrid(longitude1, latitude1)
syndata =syn.variables['Geopotential_isobaric'][:]/9.8
syndataiso1 = syndata[0,:,0,:,:]
syndataiso2 = syndata[0,:,1,:,:]
syndataiso3 = syndata[0,:,2,:,:]

lonStart1,lonEnd1,latStart1,latEnd1 = Get_Subgrid(longitude1,latitude1,[0,40],[20,60])
secData1 = Get_Area_Data(syndataiso3,lonStart1,lonEnd1,latStart1,latEnd1)
reshapeData1=Reshape_New_Data(secData1)
PCA_1=PCA_Run(reshapeData1)
transformedData1=PCA_1.transform(reshapeData1)

In [5]:
tree = dcl.construct_tree(transformedData1, k=4) 
print tree
fig = tree.plot(form='density')[0]
fig.show()

NetworkXError: Frozen graph can't be modified